https://youtu.be/0ZqeBEa_MWo?si=euGFINmBNnY8qPbn&t=149

In [ ]:
import copy

import holoviews as hv
import numpy as np
import pandas as pd
import sympy
from icecream import ic

In [ ]:
hv.extension("bokeh")

In [ ]:
spool_radius = 5.75
gear_ratio = (59.0 / 17.0) ** 2
STEP_DIVISION = 1
motor_steps_per_revolution = 100 * STEP_DIVISION
spool_circumfrence = spool_radius * 2 * np.pi
ic(spool_circumfrence)
steps_per_mm = motor_steps_per_revolution * gear_ratio / spool_circumfrence
ic(steps_per_mm)
max_rpm = 100.0
max_revs_per_second = max_rpm / 60.0
max_steps_per_second = max_revs_per_second * motor_steps_per_revolution
ic(max_steps_per_second)
max_velocity = max_steps_per_second / steps_per_mm
ic(max_velocity)
max_acceleration = 1.0
max_jerk = 1.0

In [ ]:
def s_curve(max_d):
    """Solve minimal s curve that hits parameters

    max_d: [0: max_velocity, 1: max_acceleration, 2: max_jerk]
    """
    nd = len(max_d)
    assert nd == 3
    ns = 7
    # x [0: positions, 1: velocity, 2: acceleration, 3: jerk]
    sc = [{"t":0.0, "dt": 0.0, "x": np.zeros(nd + 1), "dx": np.zeros(nd + 1)} for i in range(ns)]
    jerk_stage_sign = [1, 0, -1, 0, -1, 0, 1]
    coast_stage = 3

    dt_jerk = min(max_d[1] / max_d[2], np.sqrt(max_d[0] * 2 / max_d[2]))
    # dv_jerk = 1 / 2 * max_d[2] * dt_jerk ** 2
    # dt_acceleration = (max_d[0] - dv_jerk * 2) / max_d[1]
    dt_acceleration = (-max_d[2] * dt_jerk ** 2 + max_d[0]) / max_d[1]

    for(i, dt) in enumerate([dt_jerk, dt_acceleration, dt_jerk, 0.0, dt_jerk, dt_acceleration, dt_jerk]):
        sc[i]["dt"] = dt
        if i == 0:
            t_prev = 0
        else:
            t_prev = sc[i - 1]["t"]
        sc[i]["t"] = t_prev + sc[i]["dt"]
    
    for i in range(ns):
        if i == 0:
            x_prev = [0.0 for _ in range(nd + 1)]
        else:
            x_prev = sc[i - 1]["x"]
        # set jerk
        sc[i]["x"][3] = max_d[2] * jerk_stage_sign[i]
        sc[i]["dx"][3] = sc[i]["x"][3] - x_prev[3]
        # set d_acceleration
        sc[i]["dx"][2] = sc[i]["x"][3] * sc[i]["dt"]
        # set d_velocity
        sc[i]["dx"][1] = x_prev[2] * sc[i]["dt"] + 1 / 2 * sc[i]["x"][3] * sc[i]["dt"] ** 2
        # set d_position
        sc[i]["dx"][0] = x_prev[1] * sc[i]["dt"] + 1 / 2 * x_prev[2] * sc[i]["dt"] ** 2 + 1 / 6 * sc[i]["x"][3] * sc[i]["dt"] ** 3
        # set x
        sc[i]["x"] = x_prev + sc[i]["dx"]
    return sc
    # todo: use this to generate both full curve and minimal, 4 stage curve
    # make sanity check function

In [ ]:
def s_curve_long(sc, dt):
    # x [0: positions, 1: velocity, 2: acceleration, 3: jerk]
    nx = len(sc[0]["x"])
    state = {"t": 0.0, "x": np.zeros(nx), "p": 0.0, "v": 0.0, "a": 0.0, "j": 0.0}
    curve = [state.copy()]
    for i in range(len(sc)):
        if i > 0:
            state["x"] = sc[i - 1]["x"].copy()
        while state["t"] < sc[i]["t"]:
            state["t"] += dt
            state["x"][3] = sc[i]["x"][3]
            state["x"][2] += state["x"][3] * dt
            state["x"][1] += state["x"][2] * dt
            state["x"][0] += state["x"][1] * dt
            state["p"] = state["x"][0]
            state["v"] = state["x"][1]
            state["a"] = state["x"][2]
            state["j"] = state["x"][3]
            curve.append(state.copy())
        error = np.abs(state["x"][0] - sc[i]["x"][0])
        assert error / dt < 10
    
    return curve

In [ ]:
sc = s_curve([max_velocity, max_acceleration, max_jerk])
out = s_curve_long(sc, 0.0001)

# Full S Curve

Solve full s curve with all stages but no coasting. Any amount longer and we need to coast

In [ ]:

ic(sc[2])
ic(max_velocity)
ic(sc[6])

# Full S-Curve Symbolic TODO finish this to get stages times in a clean way

In [ ]:
p, t_j0, t_v1, t_c3, m_v, m_a, m_j = sympy.symbols("p t_j0 t_v1 t_c3 m_v m_a m_j")

In [ ]:
d = {"m_a": max_acceleration, "m_j": max_jerk}

In [ ]:
#t_j0 = m_a / m_j

In [ ]:
a_j0 = m_j * t_j0

In [ ]:
v_j0 = m_j * t_j0 ** 2 / 2

In [ ]:
p_j0 = m_j * t_j0 **3 / 6

In [ ]:
v_a1 = v_j0 + m_a * t_v1

In [ ]:
p_a1 = p_j0 + v_j0 * t_v1 + m_a * t_v1**2 / 2

In [ ]:
a_j2 = a_j0 - m_j * t_j0

In [ ]:
v_j2 = v_a1 + a_j0 * t_j0 - m_j * t_j0 ** 2 / 2

In [ ]:
sympy.solve(v_j2 - m_v, t_v1)

`t_v1 = (-m_j*t_j0**2 + m_v)/m_a` for max_velocity at coast

In [ ]:
v_j2.subs({"t_v1": (-m_j*t_j0**2 + m_v)/m_a}).subs({"m_a": max_acceleration, "m_j": max_jerk,  "m_v": max_velocity}) - max_velocity

In [ ]:
p_j2 = p_a1 + v_a1 * t_j0 + a_j0 * t_j0 ** 2 / 2 - m_j * t_j0 ** 3 / 6

In [ ]:
v_c3 = v_j2

In [ ]:
p_c3 = p_j2 + v_j2 * t_c3

In [ ]:
a_j4 = -m_j * t_j0

In [ ]:
v_j4 = v_j2 - m_j * t_j0 ** 2 / 2

In [ ]:
p_j4 = p_c3 + v_j2 * t_j0 - m_j * t_j0 ** 3 / 6

In [ ]:
v_a5 = v_j4 - m_a * t_v1

In [ ]:
p_a5 = p_j4 +  v_j4 * t_v1 - m_a * t_v1 ** 2 / 2

In [ ]:
a_j6 = -m_a + m_j * t_j0

In [ ]:
v_j6 = v_a5 - m_a * t_j0 + m_j * t_j0 ** 2 / 2

In [ ]:
p_j6 = p_a5 + v_a5 * t_j0 - m_a * t_j0 ** 2 / 2 + m_j * t_j0 ** 3 / 6

In [ ]:
[p_j6]

In [ ]:
p_j6.subs({"t_c3": 0, "t_j0": m_a / m_j, "t_v1": -m_a/m_j + m_v/m_a}).subs({"m_a": max_acceleration, "m_j": max_jerk,  "m_v": max_velocity}) - sc[6]["x"][0]

In [ ]:
sympy.solve(p_j6 - p, t_c3)

# Minimual S Curve

Solve longest all-jerk s curve with no max_acceleration portion. Any longer and we have a truncated max_acceleration stage. Any shorter and we have a truncated max_jerk stages.

In [ ]:
sc_min = s_curve([sc[0]["x"][1] * 2, max_acceleration, max_jerk])

In [ ]:
sc_long = s_curve_long(sc, 0.01)

In [ ]:
sc_long_min = s_curve_long(sc_min, 0.001)

In [ ]:
hv.Curve(sc_long, "t", "p") + hv.Curve(sc_long, "t", "v") + hv.Curve(sc_long, "t", "a") + hv.Curve(sc_long, "t", "j")

In [ ]:
hv.Curve(sc_long_min, "t", "p") + hv.Curve(sc_long_min, "t", "v") + hv.Curve(sc_long_min, "t", "a") + hv.Curve(sc_long_min, "t", "j")

In [ ]:
start = np.array([150.0, 200.0]).reshape(1,2)
end = np.array([0.0, 0.0]).reshape(1,2)

In [ ]:
def solve_coasting_s_curve(start, end):
    """Solve for coasting s-curve and ignore joint constraints"""
    
    # x [0: positions, 1: velocity, 2: acceleration, 3: jerk]

    dist = np.linalg.norm(start - end)
    
    assert dist >= sc[6]["x"][0], "path too short"
    
    out = copy.deepcopy(sc)
    
    # update coast stage (3) to fill up remaining time
    out[3]["dx"][0] = dist - sc[6]["x"][0]
    out[3]["dt"] = out[3]["dx"][0] / out[3]["x"][1]
    for i in range(3,7):
        out[i]["x"][0] = out[i-1]["x"][0] + out[i]["dx"][0]
        out[i]["t"] = out[i-1]["t"] + out[i]["dt"]
        
    return out

In [ ]:
out = s_curve_long(solve_coasting_s_curve(start, end), 0.1)

In [ ]:
hv.Curve(pd.DataFrame(out), "t", "p") + hv.Curve(pd.DataFrame(out), "t", "v")+ hv.Curve(pd.DataFrame(out), "t", "a")+ hv.Curve(pd.DataFrame(out), "t", "j")

# Mid a.k.a. Truncated Max Acceleration Curve

In [ ]:
start = np.array([9.0, 12.0]).reshape(1,2)
end = np.array([0.0, 0.0]).reshape(1,2)

In [ ]:
p, t_j0, t_v1, m_v, m_a, m_j = sympy.symbols("p t_j0 t_v1 m_v m_a m_j")

In [ ]:
d = {"m_a": max_acceleration, "m_j": max_jerk, t_v1: sc[1]["dt"], t_j0: sc[0]["dt"]}

In [ ]:
# t_j0 = m_a / m_j

In [ ]:
a_j0 = m_j * t_j0

In [ ]:
v_j0 = m_j * t_j0 ** 2 / 2

In [ ]:
p_j0 = m_j * t_j0 **3 / 6

In [ ]:
np.array([p_j0.subs(d), v_j0.subs(d), a_j0.subs(d)]) - sc[0]["x"][:3]

In [ ]:
v_a1 = v_j0 + m_a * t_v1

In [ ]:
p_a1 = p_j0 + v_j0 * t_v1 + m_a * t_v1**2 / 2

In [ ]:
np.array([p_a1.subs(d), v_a1.subs(d)]) - sc[1]["x"][:2]

In [ ]:
a_j2 = a_j0 - m_j * t_j0

In [ ]:
v_j2 = v_a1 + a_j0 * t_j0 - m_j * t_j0 ** 2 / 2

In [ ]:
p_j2 = p_a1 + v_a1 * t_j0 + a_j0 * t_j0 ** 2 / 2 - m_j * t_j0 ** 3 / 6

In [ ]:
np.array([p_j2.subs(d), v_j2.subs(d), a_j2.subs(d)]) - sc[2]["x"][:3]

In [ ]:
a_j4 = -m_j * t_j0

In [ ]:
v_j4 = v_j2 - m_j * t_j0 ** 2 / 2

In [ ]:
p_j4 = p_j2 + v_j2 * t_j0 - m_j * t_j0 ** 3 / 6

In [ ]:
np.array([p_j4.subs(d), v_j4.subs(d), a_j4.subs(d)]) - sc[4]["x"][:3]

In [ ]:
v_a5 = v_j4 - m_a * t_v1

In [ ]:
p_a5 = p_j4 +  v_j4 * t_v1 - m_a * t_v1 ** 2 / 2

In [ ]:
np.array([p_a5.subs(d), v_a5.subs(d)]) - sc[5]["x"][:2]

In [ ]:
a_j6 = -m_a + m_j * t_j0

In [ ]:
v_j6 = v_a5 - m_a * t_j0 + m_j * t_j0 ** 2 / 2

In [ ]:
p_j6 = p_a5 + v_a5 * t_j0 - m_a * t_j0 ** 2 / 2 + m_j * t_j0 ** 3 / 6

In [ ]:
[p_j6.subs({"t_v1": 0})]

The minimal length of a truncated max_acceleration (mid) s-curve is

`-m_a*t_j0**2/2 + 5*m_j*t_j0**3/2`

In [ ]:
sc_min[6]["x"][0] - (-m_a*t_j0**2/2 + 5*m_j*t_j0**3/2).subs(d)

In [ ]:
np.array([p_j6.subs(d), v_j6.subs(d), a_j6.subs(d)]) - sc[6]["x"][:3]

In [ ]:
t_v1_s = sympy.solve(p_j6 - p, t_v1)[1]

In [ ]:
t_v1_s

In [ ]:
[v_j2.subs({"t_v1": t_v1_s})]

In [ ]:
def s_curve_mid(start, end, max_d):
    # solve truncated max_acceleration curve
    # max_d: [0: max_velocity, 1: max_acceleration, 2: max_jerk]
    dist = np.linalg.norm(start - end)
    
    assert dist <= sc[6]["x"][0], "Path too long"
    assert dist >= sc_min[6]["x"][0], "Path too short"
    p = dist
    m_a = max_d[1]
    m_j = max_d[2]
    mv_s = m_a**2/m_j + (-3*m_a**2 + np.sqrt(m_a*(m_a**3 + 4*m_j**2*p)))/(2*m_j)
    ic(mv_s)
    return s_curve([mv_s, max_d[1], max_d[2]])

In [ ]:
sc_mid = s_curve_mid(start, end, [max_velocity, max_acceleration, max_jerk])

In [ ]:
sc_mid[6]["x"][0]

In [ ]:
out = s_curve_long(sc_mid, 0.01)

In [ ]:
hv.Curve(pd.DataFrame(out), "t", "p") + hv.Curve(pd.DataFrame(out), "t", "v")+ hv.Curve(pd.DataFrame(out), "t", "a")+ hv.Curve(pd.DataFrame(out), "t", "j")

# Short a.k.a. Truncated Max Jerk Curve

In [ ]:
start = np.array([0.6, 0.8]).reshape(1,2)
end = np.array([0.0, 0.0]).reshape(1,2)

In [ ]:
p, t_j0, m_v, m_a, m_j = sympy.symbols("p t_j0 m_v m_a m_j")

In [ ]:
d = {"m_a": max_acceleration, "m_j": max_jerk, t_j0: sc[0]["dt"]}

In [ ]:
a_j0 = m_j * t_j0

In [ ]:
v_j0 = m_j * t_j0 ** 2 / 2

In [ ]:
p_j0 = m_j * t_j0 **3 / 6

In [ ]:
np.array([p_j0.subs(d), v_j0.subs(d), a_j0.subs(d)]) - sc_min[0]["x"][:3]

In [ ]:
v_a1 = v_j0

In [ ]:
p_a1 = p_j0

In [ ]:
np.array([p_a1.subs(d), v_a1.subs(d)]) - sc_min[1]["x"][:2]

In [ ]:
a_j2 = a_j0 - m_j * t_j0

In [ ]:
v_j2 = v_a1 + a_j0 * t_j0 - m_j * t_j0 ** 2 / 2

In [ ]:
p_j2 = p_a1 + v_a1 * t_j0 + a_j0 * t_j0 ** 2 / 2 - m_j * t_j0 ** 3 / 6

In [ ]:
np.array([p_j2.subs(d), v_j2.subs(d), a_j2.subs(d)]) - sc_min[2]["x"][:3]

In [ ]:
a_j4 = -m_j * t_j0

In [ ]:
v_j4 = v_j2 - m_j * t_j0 ** 2 / 2

In [ ]:
p_j4 = p_j2 + v_j2 * t_j0 - m_j * t_j0 ** 3 / 6

In [ ]:
np.array([p_j4.subs(d), v_j4.subs(d), a_j4.subs(d)]) - sc_min[4]["x"][:3]

In [ ]:
v_a5 = v_j4

In [ ]:
p_a5 = p_j4

In [ ]:
np.array([p_a5.subs(d), v_a5.subs(d)]) - sc_min[5]["x"][:2]

In [ ]:
a_j6 = a_j4 + m_j * t_j0

In [ ]:
v_j6 = v_a5 + a_j4 * t_j0 + m_j * t_j0 ** 2 / 2

In [ ]:
p_j6 = p_a5 + v_a5 * t_j0 + a_j4 * t_j0 ** 2 / 2 + m_j * t_j0 ** 3 / 6

In [ ]:
np.array([p_j6.subs(d), v_j6.subs(d)]) - sc_min[6]["x"][:2]

In [ ]:
p_j6

In [ ]:
sympy.solve(p_j6 - p, t_j0)

In [ ]:
def s_curve_min(dist, max_d):
    """Solve minimal s curve that hits parameters

    max_d: [0: max_velocity, 1: max_acceleration, 2: max_jerk]
    """
    
    nd = len(max_d)
    assert nd == 3
    ns = 7
    # x [0: positions, 1: velocity, 2: acceleration, 3: jerk]
    sc = [{"t":0.0, "dt": 0.0, "x": np.zeros(nd + 1), "dx": np.zeros(nd + 1)} for i in range(ns)]
    jerk_stage_sign = [1, 0, -1, 0, -1, 0, 1]
    coast_stage = 3
    t_j0 = 2 ** (2 / 3) * (dist / max_d[2]) ** (1 / 3) / 2

    dt_jerk = min(max_d[1] / max_d[2], np.sqrt(max_d[0] * 2 / max_d[2]))
    assert t_j0 <= dt_jerk

    for(i, dt) in enumerate([t_j0, 0.0, t_j0, 0.0, t_j0, 0.0, t_j0]):
        sc[i]["dt"] = dt
        if i == 0:
            t_prev = 0
        else:
            t_prev = sc[i - 1]["t"]
        sc[i]["t"] = t_prev + sc[i]["dt"]
    
    for i in range(ns):
        if i == 0:
            x_prev = [0.0 for _ in range(nd + 1)]
        else:
            x_prev = sc[i - 1]["x"]
        # set jerk
        sc[i]["x"][3] = max_d[2] * jerk_stage_sign[i]
        sc[i]["dx"][3] = sc[i]["x"][3] - x_prev[3]
        # set d_acceleration
        sc[i]["dx"][2] = sc[i]["x"][3] * sc[i]["dt"]
        # set d_velocity
        sc[i]["dx"][1] = x_prev[2] * sc[i]["dt"] + 1 / 2 * sc[i]["x"][3] * sc[i]["dt"] ** 2
        # set d_position
        sc[i]["dx"][0] = x_prev[1] * sc[i]["dt"] + 1 / 2 * x_prev[2] * sc[i]["dt"] ** 2 + 1 / 6 * sc[i]["x"][3] * sc[i]["dt"] ** 3
        # set x
        sc[i]["x"] = x_prev + sc[i]["dx"]
    return sc
    # todo: use this to generate both full curve and minimal, 4 stage curve
    # make sanity check function

In [ ]:
out = s_curve_long(s_curve_min(1, [max_velocity, max_acceleration, max_jerk]), 0.001)

In [ ]:
hv.Curve(pd.DataFrame(out), "t", "p") + hv.Curve(pd.DataFrame(out), "t", "v")+ hv.Curve(pd.DataFrame(out), "t", "a")+ hv.Curve(pd.DataFrame(out), "t", "j")

# Motor Control

We will want the penholder to follow the s curve and will need to back out the other info for the joints. We will want to solve it for t given p_motor so we know when to make the motor step.

In [ ]:
start = np.array([0.6, 0.8]).reshape(1,2)
end = np.array([0.0, 0.0]).reshape(1,2)

In [ ]:
p, t_j0, m_v, m_a, m_j, d_x, d_y, x_m, y_m, p_m = sympy.symbols("p t_j0 m_v m_a m_j d_x d_y x_m y_m p_m")

In [ ]:
p_j0 = m_j * t_j0 **3 / 6

In [ ]:
# d_x and d_y are the basis vectors for the stright motion of p
p_j0x = p_j0 * d_x

In [ ]:
p_j0y = p_j0 * d_y

In [ ]:
solutions = sympy.solve(sympy.sqrt(((x_m - p_j0x) ** 2 + (y_m - p_j0y) ** 2)) - p_m, t_j0)

In [ ]:
len(solutions)

In [ ]:
for s in solutions:
    display(s)

In [ ]:
solutions[5:]

hmm.

6 ugly solutions. That's a nasty solution assuming I did it right. Maybe this is not the way to do it.

might be the last one

`(6*d_x*x_m/(d_x**2*m_j + d_y**2*m_j) + 6*d_y*y_m/(d_x**2*m_j + d_y**2*m_j) + sqrt(36*d_x**2*p_m**2 - 36*d_x**2*y_m**2 + 72*d_x*d_y*x_m*y_m + 36*d_y**2*p_m**2 - 36*d_y**2*x_m**2)/(d_x**2*m_j + d_y**2*m_j))**(1/3)`

# Plan B

The more straight forward way to do this is to just calculate where the motors should be in a continuous loop and update them whenever they are more than a step out of whack.

You could even estimate when that would be by computing the desired position twice. Do it once for the current time and again with a small time delta to see how much closer we are to needing to step. This would come about naturally in a agressive loop scenario if you just remember the last time you ran. What time was it and how close to a step were you then. You would even know about how long a calculation takes which could factor in. You might take the step early to avoid being late.